In [17]:
import pandas as pd
import spacy
import language_tool_python
import gensim
from gensim import models
from nltk.corpus import stopwords
import json

import tqdm 
import numpy as np
import re
import matplotlib.pyplot as plt

In [2]:
# define functions to preprocess the data
def lemmatize(text,nlp):
    # can be parallelized
    doc = nlp(text)
    lemma = [n.lemma_ for n in doc]
    return lemma

def preprocess(text,nlp):
    
    result = []
    for token in gensim.utils.simple_preprocess(text): #  gensim.utils.simple_preprocess tokenizes el texto
        token = ''.join(x for x in token.lower() if x.isalpha())
        if token not in palabrasVacias_nltk and len(token) > 2:
            result.append(token)       
    result = lemmatize(' '.join(result),nlp)
    return result

def remove_words(text):
    # Reemplazar simobolo por palabra para que no me elimine los hashtags
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b','',text, flags=re.MULTILINE) #Remove URL
    text = re.sub(r'@\w+','', text) # remove mentions
    return text

def correct_text(text):
    coincidencias = corrector.check(text)
    corrected = corrector.correct(text)
    return corrected

# load spanish language tool
corrector = language_tool_python.LanguageTool('es')
nlp = spacy.load('es_core_news_lg')

# define stopwords
palabrasVacias_nltk = stopwords.words('spanish')
palabrasVacias_nltk.append("usted")
palabrasVacias_nltk.append("uds")
palabrasVacias_nltk.append("hacer")
palabrasVacias_nltk.append("bien")
palabrasVacias_nltk.append("navidad")
palabrasVacias_nltk.append("jajaja")

In [3]:
# load the fitted LDA
lda_model = models.LdaModel.load("final_lda_model/final_lda_model.model")

# load the dictionary
dictionary_spanish = gensim.corpora.Dictionary.load("final_lda_model/final_lda_model.model.id2word")

# define the tfidf from the beginning of january (which we used to tune the LDA)
# read the old corpus and transform file
with open('final_lda_model/bow_corpus.json') as f:
   lda_corpus = [[tuple(i) for i in x] for x in json.load(f)]
tfidf = models.TfidfModel(lda_corpus)

In [5]:
# load tweet data and preprocess
tweets = pd.read_csv('data/politicians_tweets.csv')
tweets['tweet_c'] = tweets['label'].apply(lambda x: remove_words(x))
preprocessed_tweets = tweets['tweet_c'].apply(lambda x: preprocess(x, nlp))

# define the bag of word corpus for all current tweets
bow_corpus = [dictionary_spanish.doc2bow(doc) for doc in preprocessed_tweets]

In [6]:
# get topics for all tweets
tfidf_corpus = tfidf[bow_corpus]
tpcs = lda_model[tfidf_corpus]

# extract the main topic and the corresponding score
topic, score = [], []
for t in tqdm.tqdm(tpcs):
    topic.append(int(np.argmax([j[1] for j in t])))
    score.append(float(np.max([j[1] for j in t])))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1358/1358 [00:01<00:00, 1086.69it/s]


In [51]:
# load the topic names from Bibian
topic_table = pd.read_excel("data/topic_names.xlsx")
topic_table["lda_topic"] = topic_table.index
topic_table["NAME"] = [j.replace("\n",'') for j in topic_table['NAME']]
topic_lda_dict = dict(zip(topic_table["lda_topic"],topic_table["NUMBER OF TOPICS"]))
topic_label_dict = dict(zip(topic_table["NUMBER OF TOPICS"],topic_table["NAME"]))
final_topic = [topic_lda_dict[j] for j in topic]
final_label = [topic_label_dict[j] for j in final_topic]

In [57]:
# define the list of tuples
def listOfTuples(l0,l1, l2):
    return list(map(lambda w, x, y,:(w,x,y), l0,l1, l2))

topic_list = listOfTuples(tweets['id'],final_topic,score)

In [73]:
# upload data to database

import psycopg2
from sshtunnel import SSHTunnelForwarder

try:
    with SSHTunnelForwarder(
        ('161.35.123.231', 22),
        ssh_username="postgres",
        ssh_password="dbConn2021!",
        remote_bind_address=('localhost', 5432)) as server:
        
        print("server connected")
        
        keepalive_kwargs = {
            "keepalives": 1,
            "keepalives_idle": 60,
            "keepalives_interval": 10,
            "keepalives_count": 5
        } 
        
        params = {
            'database': 'tweetproject',
            'user': 'postgres',
            'password': 'padova2021',
            'host': server.local_bind_host,
            'port': server.local_bind_port,
            **keepalive_kwargs
        }
        
        conn = psycopg2.connect(**params)
        curs = conn.cursor()
        print("database connected")   
        
        sql = "INSERT INTO tweet_topic(id_tweet,id_topic,score) VALUES(%s, %s, %s)"
        curs.executemany(sql,topic_list)
        
        conn.commit()
        count = curs.rowcount
        print(count, "Record inserted successfully into tweet_topic")

except (Exception, psycopg2.Error) as error:
    print("Failed to insert record tweet_topic:", error)

server connected
database connected
1358 Record inserted successfully into tweet_topic
